In [2]:
import os
import math
import json
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input, Concatenate, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

print("TF version:", tf.__version__)

# ============================
# 1. BASIC CONFIG
# ============================

img_rows, img_cols = 48, 48
batch_size = 32

# 🔴 IMPORTANT: apna correct dataset path
BASE_DIR = r"C:\Users\ACIL\Documents\IITJ\Working\archive (2)"
train_data_dir = os.path.join(BASE_DIR, "train")
validation_data_dir = os.path.join(BASE_DIR, "test")

print("Train dir:", train_data_dir)
print("Val dir  :", validation_data_dir)

if not os.path.isdir(train_data_dir):
    raise FileNotFoundError(f"Train directory not found: {train_data_dir}")
if not os.path.isdir(validation_data_dir):
    raise FileNotFoundError(f"Validation directory not found: {validation_data_dir}")

# ============================
# 2. DATA GENERATORS
# ============================

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode="nearest",
)

validation_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    color_mode="grayscale",
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    color_mode="grayscale",
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False,
)

num_classes = train_generator.num_classes
print("Detected classes:", train_generator.class_indices)
print("num_classes =", num_classes)

# Optional: save class indices for fusion as well
with open("class_indices_fusion.json", "w") as f:
    json.dump(train_generator.class_indices, f)
print("Saved class_indices_fusion.json")

# ============================
# 3. LOAD PRETRAINED CNN & TCN
# ============================

cnn_path = r"C:\Users\ACIL\Documents\IITJ\Emotion_little_vgg.h5"
tcn_path = r"C:\Users\ACIL\Documents\IITJ\Working\emotion_tcn.h5"

print("Loading CNN from:", cnn_path)
print("Loading TCN from:", tcn_path)

cnn_model = load_model(cnn_path)
tcn_model = load_model(tcn_path)

print("CNN input shape:", cnn_model.input_shape)
print("CNN output shape:", cnn_model.output_shape)
print("TCN input shape:", tcn_model.input_shape)
print("TCN output shape:", tcn_model.output_shape)

# Freeze CNN & TCN weights (sirf fusion head train hoga)
for layer in cnn_model.layers:
    layer.trainable = False

for layer in tcn_model.layers:
    layer.trainable = False

# ============================
# 4. BUILD FUSION MODEL
# ============================

# Common input for both models
fusion_input = Input(shape=(img_rows, img_cols, 1), name="fusion_input")

# Pass same input through both pretrained models
cnn_out = cnn_model(fusion_input)   # shape: (None, num_classes)
tcn_out = tcn_model(fusion_input)   # shape: (None, num_classes)

# Concatenate their outputs (feature-level fusion)
combined = Concatenate(name="concat_cnn_tcn")([cnn_out, tcn_out])  # shape: (None, 2 * num_classes)

# Small fusion head
x = Dense(64, activation="relu", name="fusion_dense")(combined)
fusion_output = Dense(num_classes, activation="softmax", name="fusion_softmax")(x)

fusion_model = Model(inputs=fusion_input, outputs=fusion_output, name="cnn_tcn_fusion")
fusion_model.summary()

# ============================
# 5. COMPILE
# ============================

fusion_model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(learning_rate=1e-4),
    metrics=["accuracy"],
)

# ============================
# 6. CALLBACKS
# ============================

fusion_model_path = "emotion_fusion.h5"

checkpoint = ModelCheckpoint(
    fusion_model_path,
    monitor="val_loss",
    mode="min",
    save_best_only=True,
    verbose=1,
)

earlystop = EarlyStopping(
    monitor="val_loss",
    min_delta=0.0,
    patience=10,
    verbose=1,
    restore_best_weights=True,
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.2,
    patience=3,
    verbose=1,
    min_delta=1e-4,
)

callbacks = [checkpoint, earlystop, reduce_lr]

# ============================
# 7. TRAIN FUSION MODEL
# ============================

nb_train_samples = train_generator.samples
nb_validation_samples = validation_generator.samples

steps_per_epoch = math.ceil(nb_train_samples / batch_size)
val_steps = math.ceil(nb_validation_samples / batch_size)

epochs = 60

history = fusion_model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=validation_generator,
    validation_steps=val_steps,
)

# ============================
# 8. EVALUATE
# ============================

val_loss, val_acc = fusion_model.evaluate(
    validation_generator,
    steps=val_steps,
)
print(f"Fusion validation accuracy: {val_acc * 100:.2f}%")
print("Best Fusion model saved as:", fusion_model_path)


TF version: 2.20.0
Train dir: C:\Users\ACIL\Documents\IITJ\Working\archive (2)\train
Val dir  : C:\Users\ACIL\Documents\IITJ\Working\archive (2)\test
Found 24176 images belonging to 5 classes.
Found 6043 images belonging to 5 classes.
Detected classes: {'angry': 0, 'happy': 1, 'neutral': 2, 'sad': 3, 'surprise': 4}
num_classes = 5
Saved class_indices_fusion.json
Loading CNN from: C:\Users\ACIL\Documents\IITJ\Emotion_little_vgg.h5
Loading TCN from: C:\Users\ACIL\Documents\IITJ\Working\emotion_tcn.h5


CNN input shape: (None, 48, 48, 1)
CNN output shape: (None, 5)
TCN input shape: (None, 48, 48, 1)
TCN output shape: (None, 5)


Model: "cnn_tcn_fusion"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ fusion_input (InputLayer)     │ (None, 48, 48, 1)         │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ sequential_4 (Sequential)     │ (None, 5)                 │       1,328,037 │ fusion_input[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ emotion_tcn (Functional)      │ (None, 5)                 │          80,197 │ fusion_input[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concat_cnn_tcn (Concatenate)  │ (None, 10)                │               0 │ sequential_4[0][0],        │
│                               │                           │                 │ emotion_tcn[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ fusion_dense (Dense)          │ (None, 64)                │             704 │ concat_cnn_tcn[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ fusion_softmax (Dense)        │ (None, 5)                 │             325 │ fusion_dense[0][0]         │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 1,409,263 (5.38 MB)

...

756/756 ━━━━━━━━━━━━━━━━━━━━ 114s 150ms/step - accuracy: 0.6840 - loss: 0.8349 - val_accuracy: 0.6962 - val_loss: 0.8053 - learning_rate: 1.6000e-07
Epoch 38/60
756/756 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - accuracy: 0.6866 - loss: 0.8374
Epoch 38: val_loss improved from 0.80533 to 0.80533, saving model to emotion_fusion.h5


756/756 ━━━━━━━━━━━━━━━━━━━━ 115s 152ms/step - accuracy: 0.6825 - loss: 0.8392 - val_accuracy: 0.6962 - val_loss: 0.8053 - learning_rate: 1.6000e-07
Epoch 39/60
756/756 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - accuracy: 0.6838 - loss: 0.8311
Epoch 39: val_loss improved from 0.80533 to 0.80533, saving model to emotion_fusion.h5



Epoch 39: ReduceLROnPlateau reducing learning rate to 3.199999980552093e-08.
756/756 ━━━━━━━━━━━━━━━━━━━━ 112s 148ms/step - accuracy: 0.6813 - loss: 0.8374 - val_accuracy: 0.6962 - val_loss: 0.8053 - learning_rate: 1.6000e-07
Epoch 40/60
756/756 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - accuracy: 0.6820 - loss: 0.8381
Epoch 40: val_loss improved from 0.80533 to 0.80533, saving model to emotion_fusion.h5


756/756 ━━━━━━━━━━━━━━━━━━━━ 114s 151ms/step - accuracy: 0.6825 - loss: 0.8346 - val_accuracy: 0.6962 - val_loss: 0.8053 - learning_rate: 3.2000e-08
Epoch 41/60
756/756 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - accuracy: 0.6820 - loss: 0.8441
Epoch 41: val_loss improved from 0.80533 to 0.80533, saving model to emotion_fusion.h5


756/756 ━━━━━━━━━━━━━━━━━━━━ 114s 151ms/step - accuracy: 0.6855 - loss: 0.8339 - val_accuracy: 0.6962 - val_loss: 0.8053 - learning_rate: 3.2000e-08
Epoch 42/60
756/756 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - accuracy: 0.6878 - loss: 0.8247
Epoch 42: val_loss did not improve from 0.80533

Epoch 42: ReduceLROnPlateau reducing learning rate to 6.399999818995639e-09.
756/756 ━━━━━━━━━━━━━━━━━━━━ 113s 149ms/step - accuracy: 0.6839 - loss: 0.8300 - val_accuracy: 0.6962 - val_loss: 0.8053 - learning_rate: 3.2000e-08
Epoch 43/60
756/756 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - accuracy: 0.6849 - loss: 0.8386
Epoch 43: val_loss did not improve from 0.80533
756/756 ━━━━━━━━━━━━━━━━━━━━ 113s 149ms/step - accuracy: 0.6842 - loss: 0.8367 - val_accuracy: 0.6962 - val_loss: 0.8053 - learning_rate: 6.4000e-09
Epoch 44/60
756/756 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - accuracy: 0.6861 - loss: 0.8347
Epoch 44: val_loss did not improve from 0.80533
756/756 ━━━━━━━━━━━━━━━━━━━━ 113s 149ms/step - accuracy: 0.6841 